In [103]:
import pandas as pd
import numpy as np

In [104]:
xlsx = pd.ExcelFile('FracStats_JobList_Summer2017.xlsx')
df_stats = xlsx.parse('FracStats')
df_production = xlsx.parse('Production')

In [105]:
df_stats.head()

,Well,Formation,Basing,System,Fluid Type,Lateral [ft],TVD [ft],MD [ft],Stage [count],Screen Out,...,Phasing [deg],Perf Diameter [in],Perfs Open [count],Cluster Spacing [ft],Stage Spacing [ft],Perfs [area],Ball Size [in],Rate at Shift [bpm],Stage Length [ft],Well Section
0,P2,Niobrara,Denver-Julesburg,Sliding Sleeve,Gel,7104.0,6648.0,14124.0,1.0,No,...,NaN,NaN,NaN,NaN,TS,NaN,1.095,9.6,TS,Toe
1,P2,Niobrara,Denver-Julesburg,Sliding Sleeve,Gel,7104.0,6648.0,14124.0,2.0,No,...,NaN,NaN,NaN,NaN,193,NaN,1.15,11.9,193.52,Toe
2,P2,Niobrara,Denver-Julesburg,Sliding Sleeve,Gel,7104.0,6648.0,14124.0,3.0,No,...,NaN,NaN,NaN,NaN,197,NaN,1.205,11.9,196.87,Toe
3,P2,Niobrara,Denver-Julesburg,Sliding Sleeve,Gel,7104.0,6648.0,14124.0,4.0,No,...,NaN,NaN,NaN,NaN,197,NaN,1.26,11.9,197,Toe
4,P2,Niobrara,Denver-Julesburg,Sliding Sleeve,Gel,7104.0,6648.0,14124.0,5.0,No,...,NaN,NaN,NaN,NaN,197,NaN,1.315,11.8,196.77,Toe


In [106]:
df_production.head()

,Well,Days On,Day Cum,Days Cum (Zero),Gas Montlhy Prod [mcf],Oil Montlhy Prod [bbl],Water Montlhy Prod [bbl],Total HD Montlhy Production [boe],Gas Daily Prod [mcf/day],Oil Daily Prod [bbl/day],...,Water Daily Prod [bbl/day],Gas Daily Prod [boe/day],Gas Cum [mcf],Gas Cum [boe],Oil Cum [bbl],Total HD Cum Production [mcfe],Total HD Cum Production [boe],Water Cum [bbl],Total Daily Production [boe/day],Total Cum Production [boe]
0,S3,19.0,19.0,0.0,34304.0,4067.0,NaN,9784.333333,1805.473684,214.052632,...,0.000000,300.912281,34304.0,5717.333333,4067.0,58706.0,9784.333333,NaN,514.964912,9784.333333
1,S3,31.0,50.0,31.0,54267.0,11096.0,102.0,20140.500000,1750.548387,357.935484,...,3.290323,291.758065,88571.0,1849.333333,15163.0,120843.0,17012.333333,102.0,652.983871,17114.333333
2,S3,31.0,81.0,62.0,58660.0,14525.0,2343.0,24301.666667,1892.258065,468.548387,...,75.580645,315.376344,147231.0,2420.833333,29688.0,145810.0,32108.833333,2445.0,859.505376,34553.833333
3,S3,29.0,110.0,91.0,60078.0,13718.0,1719.0,23731.000000,2071.655172,473.034483,...,59.275862,345.275862,207309.0,2286.333333,43406.0,142386.0,45692.333333,4164.0,877.586207,49856.333333
4,S3,31.0,141.0,122.0,66339.0,12281.0,1555.0,23337.500000,2139.967742,396.161290,...,50.161290,356.661290,273648.0,2046.833333,55687.0,140025.0,57733.833333,5719.0,802.983871,63452.833333


In [107]:
wells_w_stats = df_stats['Well'].unique()
print(len(wells_w_stats))
print(df_stats.shape)

38
(1103, 145)


38 wells with frac stats, 1103 stages. Since production is a potential target (response) variable, let's plot up time-series production for each well. We could potentially build a model to forecast production.

In [108]:
wells_w_production = df_production['Well '].unique()
print(len(wells_w_production))

23


38 wells with stats vs. 23 wells with production data. Let's start by analyzing wells in which both data sets are available by selecting the intersection.

In [109]:
complete_well_data = set(wells_w_stats).intersection(set(wells_w_production))
print(len(complete_well_data))

22


In [110]:
print(complete_well_data)

{'S3', 'D10', 'P9', 'P4', 'P5', 'P6', 'S8', 'P3', 'D4', 'P10', 'S6', 'D5', 'S5', 'P2', 'D9', 'D1', 'P7', 'P8', 'D2', 'D3', 'P11', 'P1'}


Let's narrow our FracStats and Production data frames donwn to these wells and start exploring.

In [113]:
df1_stats = df_stats[df_stats['Well'].isin(complete_well_data)]
df1_production = df_production[df_production['Well '].isin(complete_well_data)]

In [119]:
df1_production.columns

Index(['Well ', 'Days On', 'Day Cum', 'Days Cum (Zero)',
       'Gas Montlhy Prod [mcf]', 'Oil Montlhy Prod [bbl]',
       'Water Montlhy Prod [bbl]', 'Total HD Montlhy Production  [boe]',
       'Gas Daily Prod [mcf/day]', 'Oil Daily Prod [bbl/day]',
       'Total HD Daily Production  [mcfe/day]',
       'Total HD Daily Production  [boe/day]', 'Water Daily Prod [bbl/day]',
       'Gas Daily Prod  [boe/day]', 'Gas Cum [mcf]', 'Gas Cum [boe]',
       'Oil Cum [bbl]', 'Total HD Cum Production [mcfe]',
       'Total HD Cum Production [boe]', 'Water Cum [bbl]',
       'Total Daily Production  [boe/day]', 'Total Cum Production  [boe]'],
      dtype='object')